# METHOD: description of a simulation with the Reyes probe

author: steeve.laquitaine@epfl.ch  
date: 2023.02.09  
last modified: 2023.02.09  

**Brief**:

* We simulated the activity of a mircrocircuit of the somatosensory cortex S1 (30K cells among 4.2M)
* The circuit structure is the one of the entire 4.2M cells of the somatosensory cortex and is configured in project 83, 2020-08-05 (`/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805`)
    * its circuit and simulation parameters and paths to its files are stored in the CircuitConfig (also called BlueConfig) file in `/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/CircuitConfig`
        * see sections' description here (4)
* Opening the BlueConfig of one piece of the simulation that was used to generate the simulation with BluePy gives access to information about the entire S1 circuit structure. Note that we can have one single piece for short sinmulations or many for a campaign (run with BBP_workflow).
* LFP sampling frequency: `20 KHz`
* Probe contacts: `128`

## Setup

Setup Jupyter notebook's kernel based on python env setup with requirements_silico.txt (see comments in file for setup)

In [10]:
# SETUP PACKAGES
%load_ext autoreload
%autoreload 2
import os
import pprint as pp
import pandas as pd
import yaml
import numpy as np
pd.set_option('display.max_colwidth', None)

# set project path
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting"

os.chdir(PROJ_PATH)
from src.nodes.io.silico.loading import load_campaign_params
from src.nodes.utils import get_config
from src.nodes.io.silico import loading_raw_recording
from src.nodes.truth.silico import ground_truth

# SETUP PARAMETERS
EXPERIMENT = "supp/silico_reyes"    # specifies the experiment 
SIMULATION_DATE = "2023_01_13" # specifies the run (date)


# SETUP CONFIG
print("CONFIG:\n")
data_conf, param_conf = get_config(EXPERIMENT, SIMULATION_DATE).values()
pp.pprint(data_conf)
pp.pprint(param_conf)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CONFIG:

2023-02-09 16:51:30,172 - root - utils.py - get_config - INFO - Reading experiment config.
2023-02-09 16:51:30,254 - root - utils.py - get_config - INFO - Reading experiment config. - done
{'dataeng': {'blueconfig': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/raw/reyes_probe_lfp_4sec_2023_01_11/3/BlueConfig',
             'campaign': {'input': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/dataeng/0_silico/reyes_probe_lfp_4sec_2023_01_11/simulations/',
                          'output': {'spike_file_path': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/dataeng/0_silico/reyes_probe_lfp_4sec_2023_01_11/campaign/raw/spiketrains.pkl',
                                     'trace_file_path': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/dataeng/0_silico/reyes_probe_lfp_4sec_2023_01_11/campaign/raw/traces.pkl',
                                     'unit_trace_path': '/gpfs/bbp

## Load a simulation

Load the simulation parameters from that piece

In [5]:
# load parameters from the first simulation "BlueConfig" file
simulation_params = load_campaign_params(data_conf)

/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:51: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  Bool8 = np.bool8
/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:54: DeprecationWarning: `np.object0` is a deprecated alias for ``np.object0` is a deprecated alias for `np.object_`. `object` can be used instead.  (Deprecated NumPy 1.24)`.  (Deprecated NumPy 1.24)
  Object0 = np.object0
/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:66: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  Int0 = np.int0
/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/env_silico/lib/python3.9/site-packages/nptyping/typing_.py:80: DeprecationWarning: `np.uint0` is a deprecated alias for `np.uin

In [6]:
# list simulation parameters
blueconfig_path = simulation_params["paths"]["BlueConfig_path"]

# describe the content of the first simulation piece's circuit config file (also BlueConfig) file
print("Used BlueConfig path:", blueconfig_path)
os.system(f"""cat {blueconfig_path}""")

Used BlueConfig path: /gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/raw/reyes_probe_lfp_4sec_2023_01_11/3/BlueConfig
Run Default
{


  CircuitPath /gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805
      nrnPath /gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/connectome/functional/edges.sonata
    CellLibraryFile /gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/circuit.mvd3

      BioName /gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/bioname
      Atlas /gpfs/bbp.cscs.ch/project/proj83/data/atlas/S1/MEAN/P14-MEAN

      METypePath /gpfs/bbp.cscs.ch/project/proj45/scratch/S1full/METypes
      MEComboInfoFile /gpfs/bbp.cscs.ch/project/proj83/singlecell/fixed_L6_allBPC_thresholds/mecombo_emodel.tsv
      MorphologyPath /gpfs/bbp.cscs.ch/project/proj83/morphologies/fixed_ais_L23PC_20201210/ascii
      MorphologyType asc


    RNGMode Random123
    BaseSeed 3

      OutputRoot .
      TargetFile /gpfs/bbp.cscs.ch/project/proj68/scratch/tharayil/forAlain

0

lusInject RelativeShotNoise_L1I_inject
{
    Stimulus RelativeShotNoise_L1I
    Target Layer1
}

StimulusInject RelativeShotNoise_L23E_inject
{
    Stimulus RelativeShotNoise_L23E
    Target Layer23Excitatory
}

StimulusInject RelativeShotNoise_L23I_inject
{
    Stimulus RelativeShotNoise_L23I
    Target Layer23Inhibitory
}

StimulusInject RelativeShotNoise_L4E_inject
{
    Stimulus RelativeShotNoise_L4E
    Target Layer4Excitatory
}

StimulusInject RelativeShotNoise_L4I_inject
{
    Stimulus RelativeShotNoise_L4I
    Target Layer4Inhibitory
}

StimulusInject RelativeShotNoise_L5E_inject
{
    Stimulus RelativeShotNoise_L5E
    Target Layer5Excitatory
}

StimulusInject RelativeShotNoise_L5I_inject
{
    Stimulus RelativeShotNoise_L5I
    Target Layer5Inhibitory
}

StimulusInject RelativeShotNoise_L6E_inject
{
    Stimulus RelativeShotNoise_L6E
    Target Layer6Excitatory
}

StimulusInject RelativeShotNoise_L6I_inject
{
    Stimulus RelativeShotNoise_L6I
    Target Layer6Inhibitory
}




In [7]:
# list simulation parameters
pp.pprint(simulation_params)

{'blue_config': BlueConfigFile([('Run_Default',
                                 BlueConfSectionRun([('CircuitPath',
                                                      '/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805'),
                                                     ('nrnPath',
                                                      '/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/connectome/functional/edges.sonata'),
                                                     ('CellLibraryFile',
                                                      '/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/circuit.mvd3'),
                                                     ('BioName',
                                                      '/gpfs/bbp.cscs.ch/project/proj83/circuits/Bio_M/20200805/bioname'),
                                                     ('Atlas',
                                                      '/gpfs/bbp.cscs.ch/project/proj83/data/atlas/S1

In [8]:
# list the path used for the simulation
print("simulation file paths:\n\n", simulation_params["paths"])

simulation file paths:

 {'BlueConfig_path': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/raw/reyes_probe_lfp_4sec_2023_01_11/3/BlueConfig'}


In [18]:
# NOTE: Campaign simulations are deleted after they are stacked into one campaign file to save space
# print(f"""campaign paths: {data_conf["dataeng"]["campaign"]}""")
# print("\ncampaign simulation ids found are:", os.listdir(data_conf["dataeng"]["campaign"]["input"]))
# print("\nrecording sites (count):", simulation_params["n_sites"])

campaign paths: {'input': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/dataeng/0_silico/reyes_probe_lfp_4sec_2023_01_11/simulations/', 'output': {'spike_file_path': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/dataeng/0_silico/reyes_probe_lfp_4sec_2023_01_11/campaign/raw/spiketrains.pkl', 'trace_file_path': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/dataeng/0_silico/reyes_probe_lfp_4sec_2023_01_11/campaign/raw/traces.pkl', 'unit_trace_path': '/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/dataeng/0_silico/reyes_probe_lfp_4sec_2023_01_11/campaign/raw/cells/'}}

campaign simulation ids found are: ['7', '6', '3', '5', '2', '4', '1', '9', '10', '8']

recording sites (count): 128


## Describe the circuit

The static structure of the network (see 2,3):

* `circuit.mvd3`: the circuit file
* `start.target`: ASCII file defining the Cell/Compartment/Section targets

In [19]:
simulation_params.keys()

dict_keys(['blue_config', 'paths', 'lfp_sampling_freq', 'spike_sampling_freq', 'n_sites', 'circuit'])

In [20]:
print("Circuit attributes:\n")
print(dir(simulation_params["circuit"]))

Circuit attributes:

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_config', '_projections', 'atlas', 'cells', 'config', 'connectome', 'emodels', 'morph', 'projection', 'stats', 'subcellular']


### Describe its cells

In [21]:
# report the cell object attributes
print("Cells attributes:\n\n ", dir(simulation_params["circuit"].cells))

Cells attributes:

  ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_impl', '_index_url', '_targets', 'available_properties', 'count', 'etypes', 'get', 'ids', 'mtypes', 'orientations', 'positions', 'spatial_index', 'targets']


In [10]:
# report some circuit info
print("Circuit info:\n")
print("- cell (count): ", simulation_params["circuit"].cells.count())
print("- cell global identifiers (gids): ", simulation_params["circuit"].cells.ids())

Circuit info:

- cell (count):  4234929
- cell global identifiers (gids):  [      1       2       3 ... 4234927 4234928 4234929]


In [22]:
# report the cell properties
cell_properties = simulation_params["circuit"].cells.available_properties
print("Cell properties:\n\n ", list(cell_properties))

Cell properties:

  ['exc_mini_frequency', 'me_combo', 'morphology', 'layer', 'morph_class', 'x', 'synapse_class', 'etype', 'y', 'z', 'orientation', 'inh_mini_frequency', 'mtype', 'region']


In [23]:
print("Cell properties:\n\n")
simulation_params["circuit"].cells.get(
    properties=[
        "x",
        "y",
        "z",
        "etype",
        "exc_mini_frequency",
        "inh_mini_frequency",
        "layer",
        "me_combo",
        "morph_class",
        "morphology",
        "mtype",
        "orientation",
        "region",
        "synapse_class",
    ]
).head(3)

Cell properties:




,x,y,z,etype,exc_mini_frequency,inh_mini_frequency,layer,me_combo,morph_class,morphology,mtype,orientation,region,synapse_class
1,4877.122429,-668.324217,-1696.124733,cNAC,0.010108,0.233243,1,cNAC_L23NGC_L1_DAC_1_C270106C_-_Scale_x1.000_y1.050_z1.000,INT,C270106C_-_Scale_x1.000_y1.050_z1.000,L1_DAC,"[[0.6959961932426957, 0.6180960655330769, -0.36544021777057667], [-0.3559836626278208, -0.14496710934591028, -0.923179380808504], [-0.6235903550679389, 0.772620081916346, 0.11913554501415904]]",S1FL,INH
2,5383.917646,-546.443089,-2318.223122,bNAC,0.010108,0.233243,1,bNAC_L23NGC_L1_DAC_1_sm080930a1-5_idC_-_Scale_x1.000_y1.050_z1.000_-_Clone_1,INT,sm080930a1-5_idC_-_Scale_x1.000_y1.050_z1.000_-_Clone_1,L1_DAC,"[[0.6271665074056754, 0.756419845425081, 0.18571803745372562], [0.45452886926840885, -0.16180503615058694, -0.8759124598256888], [-0.6325074537032339, 0.633757167788581, -0.4452933564364594]]",S1ULp,INH
3,5038.520009,-2747.467238,-2582.717297,cNAC,0.010108,0.233243,1,cNAC_L23NGC_L1_DAC_1_C270106C_-_Scale_x1.000_y1.025_z1.000,INT,C270106C_-_Scale_x1.000_y1.025_z1.000,L1_DAC,"[[0.3978350345973204, 0.7888446215139444, -0.46845645299800354], [-0.32797221510446284, -0.35458167330677237, -0.8756175324160895], [-0.8568322539251314, 0.5019920318725102, 0.11765410562359435]]",S1J,INH


naming legend: [`c`:classical |`b`:bursting | `d`:delayed] - [`AC`:accomodating | `NAC`:non-accomodating | `STUT`:stuttering | `IR`: irregular spiking]

* `dNAC`: delayed non-accomodating cells
* `cNAC`: classical non-accomodating cells
* `bSTUT`: bursting stuttering
* `bNAC`: bursting non-accomodating cells
* `cIR`: classical irregular spiking
* `cACint`: classical accomodating interneuron?
* `bAC`: bursting accomodating
* `cADpyr`: classical accomodating delayed pyramidal?
* `dSTUT`: delayed stuttering
* `cSTUT`: classical stuttering
* `bIR`: bursting irrelgular spiking

(see 7, 8)

In [24]:
categorical_properties = [
    "region",
    "layer",
    "etype",
    "morph_class",
    "synapse_class",
    "mtype",
    "me_combo",
    "morphology",
]
pd.DataFrame(
    simulation_params["circuit"]
    .cells.get(properties=categorical_properties)
    .apply(set),
    columns=["unique values"],
)

unique values
region                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## Describe the campaign data

In [25]:
raw_trace = loading_raw_recording.load(data_conf, param_conf)
type(raw_trace)

spikeinterface.core.numpyextractors.NumpyRecording

In [26]:
print("channel (count):", raw_trace.get_num_channels())
print("trace data dimensions:", raw_trace.get_traces().shape)
print("preview:")
raw_trace.get_traces()

channel (count): 128
trace data dimensions: (799900, 128)
preview:


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.8548499 ,  0.926285  ,  0.96943676, ...,  0.68992776,
         0.5257821 ,  0.3802115 ],
       [ 1.7177376 ,  1.8322605 ,  1.9171326 , ...,  0.46070808,
         0.11974214, -0.18930997],
       ...,
       [11.720507  , 11.324211  , 10.725289  , ..., -2.4860332 ,
        -4.18072   , -5.9242415 ],
       [11.720923  , 11.301833  , 10.691481  , ..., -2.4801247 ,
        -4.176113  , -5.9199843 ],
       [11.703304  , 11.273225  , 10.65614   , ..., -2.4411218 ,
        -4.142916  , -5.89059   ]], dtype=float32)

In [27]:
SortingExtractor = ground_truth.run(simulation_params, data_conf, param_conf)
SortingExtractor = SortingExtractor["ground_truth_sorting_object"]
print("unit (count): ", SortingExtractor.get_num_units())
print("unit (id): ", SortingExtractor.unit_ids)
print("all spike trains: ", SortingExtractor.get_all_spike_trains())

unit (count):  1221
unit (id):  [  10250   20590   47971 ... 4215563 4227733 4234649]
all spike trains:  [(array([   596,    699,    804, ..., 799726, 799844, 799871]), array([3558344, 2222288, 2566538, ..., 3972314, 3866316, 3767034]))]


# References

(1) https://bbpgitlab.epfl.ch/nse/bluepy/-/tree/lfp-reports/doc/source 

(2) https://bbpgitlab.epfl.ch/nse/bluepy/-/blob/lfp-reports/doc/source/circuit.rst 

(3) https://sonata-extension.readthedocs.io/en/latest/index.html

(4) https://sonata-extension.readthedocs.io/en/latest/blueconfig.html 

(5) https://www.edx.org/course/simulating-a-hippocampus-microcircuit 

(6) https://bbpgitlab.epfl.ch/nse/bluepy/-/tree/main/examples/ipython_notebooks 

(7) https://bbpteam.epfl.ch/project/spaces/display/MEETINSILICO/electrical+cell+models 

(8) Muralidhar, Wang, and Markram, “Synaptic and Cellular Organization of Layer 1 of the Developing Rat Somatosensory Cortex.”